In [1]:
# get the predictive interval of cumulative cases and deaths
import covasim as cv
import sys
sys.path.append('./')

sims = cv.load('fig1a.sims')

Covasim 2.1.2 (2021-03-31) — © 2021 by IDM


In [2]:
import pandas as pd

pars = pd.DataFrame([
    s.pars for s in sims
])
pars['rowidx'] = [i for i in range(len(pars))]
res = []
for i,s in enumerate(sims):
    res.append({
        'cum_diagnoses':s.results['cum_diagnoses'].values[-1],
        'cum_deaths':s.results['cum_deaths'].values[-1],
        'rowidx':i
    })
res = pd.DataFrame(res).merge(pars,on='rowidx')
res

,cum_diagnoses,cum_deaths,rowidx,pop_size,pop_infected,pop_type,location,start_day,end_day,n_days,...,prog_by_age,prognoses,interventions,analyzers,timelimit,stopping_func,n_beds_hosp,n_beds_icu,no_hosp_factor,no_icu_factor
0,8709.738932,603.791276,0,225000,300,synthpops,None,2020-01-27,2020-06-08,133,...,True,"{'age_cutoffs': [0, 10, 20, 30, 40, 50, 65, 70...","[cv.test_num(daily_tests={""1580083200000"":5.0,...",[<covasim.analysis.age_histogram at 0x12e2f5ed...,None,None,None,None,2.0,2.0
1,8481.418156,614.912877,1,225000,300,synthpops,None,2020-01-27,2020-06-08,133,...,True,"{'age_cutoffs': [0, 10, 20, 30, 40, 50, 65, 70...","[cv.test_num(daily_tests={""1580083200000"":5.0,...",[<covasim.analysis.age_histogram at 0x12e3301c...,None,None,None,None,2.0,2.0
2,8558.517383,555.033002,2,225000,300,synthpops,None,2020-01-27,2020-06-08,133,...,True,"{'age_cutoffs': [0, 10, 20, 30, 40, 50, 65, 70...","[cv.test_num(daily_tests={""1580083200000"":5.0,...",[<covasim.analysis.age_histogram at 0x12e35a47...,None,None,None,None,2.0,2.0
3,8464.684985,614.466723,3,225000,300,synthpops,None,2020-01-27,2020-06-08,133,...,True,"{'age_cutoffs': [0, 10, 20, 30, 40, 50, 65, 70...","[cv.test_num(daily_tests={""1580083200000"":5.0,...",[<covasim.analysis.age_histogram at 0x12e38876...,None,None,None,None,2.0,2.0
4,8405.378811,598.190005,4,225000,300,synthpops,None,2020-01-27,2020-06-08,133,...,True,"{'age_cutoffs': [0, 10, 20, 30, 40, 50, 65, 70...","[cv.test_num(daily_tests={""1580083200000"":5.0,...",[<covasim.analysis.age_histogram at 0x12e3aaa1...,None,None,None,None,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,8796.608883,639.294266,195,225000,300,synthpops,None,2020-01-27,2020-06-08,133,...,True,"{'age_cutoffs': [0, 10, 20, 30, 40, 50, 65, 70...","[cv.test_num(daily_tests={""1580083200000"":5.0,...",[<covasim.analysis.age_histogram at 0x135f8e6b...,None,None,None,None,2.0,2.0
196,9353.056031,630.357144,196,225000,300,synthpops,None,2020-01-27,2020-06-08,133,...,True,"{'age_cutoffs': [0, 10, 20, 30, 40, 50, 65, 70...","[cv.test_num(daily_tests={""1580083200000"":5.0,...",[<covasim.analysis.age_histogram at 0x135fc09a...,None,None,None,None,2.0,2.0
197,8915.899308,550.763025,197,225000,300,synthpops,None,2020-01-27,2020-06-08,133,...,True,"{'age_cutoffs': [0, 10, 20, 30, 40, 50, 65, 70...","[cv.test_num(daily_tests={""1580083200000"":5.0,...",[<covasim.analysis.age_histogram at 0x135fdec5...,None,None,None,None,2.0,2.0
198,8561.875296,605.366018,198,225000,300,synthpops,None,2020-01-27,2020-06-08,133,...,True,"{'age_cutoffs': [0, 10, 20, 30, 40, 50, 65, 70...","[cv.test_num(daily_tests={""1580083200000"":5.0,...",[<covasim.analysis.age_histogram at 0x13600cf4...,None,None,None,None,2.0,2.0


In [3]:
agg = (res
[['cum_diagnoses','cum_deaths']]
.describe(percentiles=[0.025,0.5,0.975])
.reset_index().rename({'index':'statistic'},axis='columns'))
agg.to_csv('calibration_summary_stats.csv',index=False)

In [4]:
agg

,statistic,cum_diagnoses,cum_deaths
0,count,200.000000,200.000000
1,mean,8511.840237,595.941817
2,std,475.720721,38.687097
3,min,7553.940708,510.140352
4,2.5%,7701.898679,530.340868
5,50%,8517.277369,595.228861
6,97.5%,9362.101762,673.942670
7,max,9611.213225,705.082647


In [5]:
import numpy as np
def roll(x):
    return pd.Series(x).rolling(window=7).mean().fillna(0).values.tolist()
def sims_to_df():
    outputs = ['cum_diagnoses','cum_deaths','new_diagnoses','new_deaths']
    d = {col: [] for  col in outputs}
    for o in outputs:
        d[o+'_data'] = []
    d['t'] = []
    d['index'] = []
    for i, s in enumerate(sims):
        for col in outputs:
            res_col = col
            data_col = col + '_data'
            if 'new_' in col:
                d[res_col]  += roll(s.results[col].values)
                d[data_col] += roll(s.data[col].values)
            else:
                d[res_col]  += s.results[col].values.tolist()
                d[data_col] += s.data[col].values.tolist()
        t = np.arange(len(s.results[col].values)).tolist()
        d['t']+= t
        d['index'] += ([i]*len(t))
    df = pd.DataFrame(d).rename(mapper = lambda x: x.replace('new_',''),axis='columns')
    return df
sims_df = sims_to_df()
outputs = ['cum_diagnoses','cum_deaths','diagnoses','deaths']
for o in outputs:
    sims_df[o+'_mse'] = (sims_df[o] - sims_df[o+'_data'])**2
mse_cols = [col for col in sims_df.columns if '_mse' in col]
sims_rmse = (sims_df.groupby('index')[mse_cols].mean()**0.5).reset_index()
sims_df.to_csv('sim_results.csv',index=False)
sims_rmse.to_csv('sim_rmse.csv',index=False)
    



In [10]:
# mae
mae_keys = []
for output in ['diagnoses','deaths']:
    col = output + '_mae'
    sims_df[output + '_mae'] = np.abs(sims_df[output] - sims_df[output + '_data'])
    mae_keys.append(col)
sims_df.groupby('index')[mae_keys].mean().describe()
sims_df.groupby('index')[mae_keys].mean().reset_index().to_csv('sim_mae.csv')

In [7]:
# corr
def get_corr(col):
    sim_corrs = (sims_df
    .groupby('index')[
        [col,f'{col}_data']
    ].corr().reset_index()).drop_duplicates(subset=['index'],keep='first')
    return sim_corrs[f'{col}_data'].describe().to_frame()
df_corr = pd.concat([get_corr('diagnoses'),get_corr('deaths')],axis=1)
df_corr

,diagnoses_data,deaths_data
count,200.000000,200.000000
mean,0.976735,0.874729
std,0.008332,0.038242
min,0.946284,0.761884
25%,0.972390,0.848497
50%,0.978225,0.883189
75%,0.982911,0.903198
max,0.993692,0.964285
